# Folders setup

In [4]:
models_folder = './step_by_step_models'
ori_filename = models_folder + '/BNN_BED_classifier__best_mean_F1=8757__QONNX.onnx'

# Model Clean

In [5]:
from finn.util.visualization import showSrc, showInNetron
from qonnx.util.cleanup import cleanup as qonnx_cleanup

In [6]:
qonnx_clean_filename = models_folder + '/01_clean.onnx'
qonnx_cleanup(ori_filename, out_file=qonnx_clean_filename)

In [7]:
showInNetron(ori_filename)

Serving './step_by_step_models/BNN_BED_classifier__best_mean_F1=8757__QONNX.onnx' at http://0.0.0.0:8083


In [8]:
showInNetron(qonnx_clean_filename)

Stopping http://0.0.0.0:8083
Serving './step_by_step_models/01_clean.onnx' at http://0.0.0.0:8083


# Dummy Input

In [9]:
from qonnx.core.modelwrapper import ModelWrapper
import qonnx.core.onnx_exec as oxe

import numpy as np

In [10]:
test_ip = np.random.randint(low=0, high=256, size=(1, 3, 224, 224)) / 255.
test_ip = test_ip.astype(np.float32)

In [11]:
clean_model = ModelWrapper(qonnx_clean_filename)

In [12]:
input_dict = {"global_in": test_ip}
output_dict = oxe.execute_onnx(clean_model, input_dict)
produced_clean_qonnx = output_dict[list(output_dict.keys())[0]]
produced_clean_qonnx

array([[-4.270301 , -6.0525727]], dtype=float32)

# Convert to FINN

In [13]:
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.fold_constants import FoldConstants
from qonnx.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs

In [14]:
model = ModelWrapper(qonnx_clean_filename)
model = model.transform(ConvertQONNXtoFINN())
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(RemoveStaticGraphInputs())

In [15]:
finn_tidy = models_folder + '/02_finn_tidy.onnx'
model.save(finn_tidy)

In [16]:
showInNetron(finn_tidy)

Stopping http://0.0.0.0:8083
Serving './step_by_step_models/02_finn_tidy.onnx' at http://0.0.0.0:8083


# PreProcess

In [17]:
import torch
from finn.util.pytorch import ToTensor
from qonnx.transformation.merge_onnx_models import MergeONNXModels
from qonnx.core.datatype import DataType
from brevitas.export import export_qonnx

In [18]:
model = ModelWrapper(finn_tidy)
global_inp_name = model.graph.input[0].name
ishape = model.get_tensor_shape(global_inp_name)
# preprocessing: torchvision's ToTensor divides uint8 inputs by 255
totensor_pyt = ToTensor()
chkpt_preproc_name = models_folder + "/prepro_node.onnx"
export_qonnx(totensor_pyt, torch.randn(ishape), chkpt_preproc_name)
qonnx_cleanup(chkpt_preproc_name, out_file=chkpt_preproc_name)
pre_model = ModelWrapper(chkpt_preproc_name)
pre_model = pre_model.transform(ConvertQONNXtoFINN())

# join preprocessing and core model
model = model.transform(MergeONNXModels(pre_model))
# add input quantization annotation: UINT8 for all BNN-PYNQ models
global_inp_name = model.graph.input[0].name
model.set_tensor_datatype(global_inp_name, DataType["UINT8"])

/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/transformation/merge_onnx_models.py:70: UserWarning: [MergeONNXModels] opsets for models to merge differ: 14 vs 17, output model will use opset 17
  warnings.warn(
/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/transformation/infer_data_layouts.py:127: UserWarning: Assuming 4D input is NCHW
  warnings.warn("Assuming 4D input is NCHW")


In [19]:
from qonnx.transformation.infer_datatypes import InferDataTypes

### Save prepro after tidy

In [20]:
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(InferDataTypes())
model = model.transform(RemoveStaticGraphInputs())

In [21]:
finn_prepro = models_folder + '/03_finn_prepro.onnx'
model.save(finn_prepro)

In [22]:
showInNetron(finn_prepro)

Stopping http://0.0.0.0:8083
Serving './step_by_step_models/03_finn_prepro.onnx' at http://0.0.0.0:8083


# Streamline

In [87]:
from qonnx.transformation.lower_convs_to_matmul import LowerConvsToMatMul
from qonnx.transformation.bipolar_to_xnor import ConvertBipolarMatMulToXnorPopcount
from finn.transformation.streamline.round_thresholds import RoundAndClipThresholds

from qonnx.transformation.change_datalayout import ChangeDataLayoutQuantAvgPool2d
from qonnx.transformation.infer_data_layouts import InferDataLayouts
from qonnx.transformation.general import RemoveUnusedTensors

from finn.transformation.streamline import Streamline
import finn.transformation.streamline.absorb as absorb
from finn.transformation.streamline.reorder import MoveScalarLinearPastInvariants
from finn.transformation.streamline.reorder import MakeMaxPoolNHWC

In [88]:
# model = ModelWrapper(finn_prepro)
# model = model.transform(MoveScalarLinearPastInvariants())
# model = model.transform(Streamline())
# model = model.transform(LowerConvsToMatMul())
# model = model.transform(MakeMaxPoolNHWC())
# model = model.transform(ChangeDataLayoutQuantAvgPool2d())
# model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
# model = model.transform(ConvertBipolarMatMulToXnorPopcount())
# model = model.transform(RoundAndClipThresholds())

# model = model.transform(Streamline())
# model = model.transform(InferDataLayouts())
# model = model.transform(RemoveUnusedTensors())

In [107]:
from finn.transformation.streamline.round_thresholds import RoundAndClipThresholds
from finn.transformation.streamline.sign_to_thres import ConvertSignToThres
from qonnx.transformation.batchnorm_to_affine import BatchNormToAffine

from qonnx.transformation.general import (
    ConvertSubToAdd,
    ConvertDivToMul,
    GiveReadableTensorNames,
    GiveUniqueNodeNames,
    SortGraph,
    RemoveUnusedTensors,
    GiveUniqueParameterTensors,
    RemoveStaticGraphInputs,
    ApplyConfig,
)

from finn.transformation.streamline.absorb import (
    AbsorbAddIntoMultiThreshold,
    AbsorbMulIntoMultiThreshold,
    FactorOutMulSignMagnitude,
    Absorb1BitMulIntoMatMul,
    Absorb1BitMulIntoConv,
    AbsorbConsecutiveTransposes,
    AbsorbTransposeIntoMultiThreshold,
)

from finn.transformation.streamline.collapse_repeated import (
    CollapseRepeatedAdd,
    CollapseRepeatedMul,
)

from finn.transformation.streamline.reorder import (
    MoveAddPastMul,
    MoveScalarMulPastMatMul,
    MoveScalarAddPastMatMul,
    MoveAddPastConv,
    MoveScalarMulPastConv,
    MoveScalarLinearPastInvariants,
    MoveMaxPoolPastMultiThreshold,
)

from qonnx.transformation.remove import RemoveIdentityOps


In [119]:
def step_resnet50_streamline_linear(model: ModelWrapper):

    streamline_transformations = [
        ConvertSubToAdd(),
        ConvertDivToMul(),
        RemoveIdentityOps(),
        CollapseRepeatedMul(),
        BatchNormToAffine(),
        ConvertSignToThres(),
        MoveAddPastMul(),
        MoveScalarAddPastMatMul(),
        MoveAddPastConv(),
        MoveScalarMulPastMatMul(),
        MoveScalarMulPastConv(),
        MoveScalarLinearPastInvariants(),
        MoveAddPastMul(),
        CollapseRepeatedAdd(),
        CollapseRepeatedMul(),
        AbsorbAddIntoMultiThreshold(),
        FactorOutMulSignMagnitude(),
        MoveMaxPoolPastMultiThreshold(),
        AbsorbMulIntoMultiThreshold(),
        Absorb1BitMulIntoMatMul(),
        Absorb1BitMulIntoConv(),
        RoundAndClipThresholds(),
        ]
    for trn in streamline_transformations:
        model = model.transform(trn)
        model = model.transform(GiveUniqueNodeNames())

    return model

In [120]:
# finn_streamline = models_folder + '/04_finn_streamline.onnx'
# model.save(finn_streamline)

In [121]:
# showInNetron(finn_streamline)

### Non Linear

In [122]:
# just for not linear
from finn.transformation.streamline.reorder import (
    MoveLinearPastEltwiseAdd,
    MoveLinearPastFork,
)

In [123]:
# model = ModelWrapper(finn_streamline)

In [124]:
def step_resnet50_streamline_nonlinear(model: ModelWrapper):
    streamline_transformations = [
        MoveLinearPastEltwiseAdd(),
        MoveLinearPastFork(),
    ]
    for trn in streamline_transformations:
        model = model.transform(trn)
        model = model.transform(GiveUniqueNodeNames())
    return model

In [125]:
# finn_streamline_non_linear = models_folder + '/05_finn_streamline_non_linear.onnx'
# model.save(finn_streamline_non_linear)

In [126]:
# showInNetron(finn_streamline_non_linear)

# Streamline Loop

In [130]:
from qonnx.transformation.double_to_single_float import DoubleToSingleFloat

In [131]:
def step_resnet50_streamline(model: ModelWrapper):
    for iter_id in range(3):
        model = step_resnet50_streamline_linear(model)
        model = step_resnet50_streamline_nonlinear(model)

        # big loop tidy up
        model = model.transform(RemoveUnusedTensors())
        model = model.transform(GiveReadableTensorNames())
        model = model.transform(InferDataTypes())
        model = model.transform(SortGraph())

    model = model.transform(DoubleToSingleFloat())

    return model

In [132]:
model = ModelWrapper(finn_prepro)

In [133]:
model = step_resnet50_streamline(model)

In [134]:
finn_streamline = models_folder + '/04_finn_streamline.onnx'
model.save(finn_streamline)

In [135]:
showInNetron(finn_streamline)

Stopping http://0.0.0.0:8083
Serving './step_by_step_models/04_finn_streamline.onnx' at http://0.0.0.0:8083


# HW Layers

In [27]:
from finn.util.basic import pynq_part_map
# change this if you have a different PYNQ board, see list above
pynq_board = "Pynq-Z1"
fpga_part = pynq_part_map[pynq_board]
target_clk_ns = 10

In [28]:
print(pynq_part_map)
print(fpga_part)

{'Ultra96': 'xczu3eg-sbva484-1-e', 'Ultra96-V2': 'xczu3eg-sbva484-1-i', 'Pynq-Z1': 'xc7z020clg400-1', 'Pynq-Z2': 'xc7z020clg400-1', 'ZCU102': 'xczu9eg-ffvb1156-2-e', 'ZCU104': 'xczu7ev-ffvc1156-2-e', 'ZCU111': 'xczu28dr-ffvg1517-2-e', 'RFSoC2x2': 'xczu28dr-ffvg1517-2-e', 'RFSoC4x2': 'xczu48dr-ffvg1517-2-e', 'KV260_SOM': 'xck26-sfvc784-2LV-c'}
xc7z020clg400-1


In [137]:
import finn.transformation.fpgadataflow.convert_to_hw_layers as to_hw
from finn.transformation.fpgadataflow.create_dataflow_partition import (
    CreateDataflowPartition,
)
from finn.transformation.move_reshape import RemoveCNVtoFCFlatten

from qonnx.custom_op.registry import getCustomOp

### Change last Bipolar Node to Binary

In [147]:
model = ModelWrapper(finn_streamline)

In [148]:
Multithreshold_node = model.get_nodes_by_op_type("MultiThreshold") 

In [149]:
for node in Multithreshold_node:
    if node.name == "MultiThreshold_24":
        node_inst = getCustomOp(node)
        if node_inst.get_nodeattr("out_dtype") == "BIPOLAR":
            node_inst.set_nodeattr("out_dtype", "BINARY")
            node_inst.set_nodeattr("out_scale", 1.0)
            node_inst.set_nodeattr("out_bias", 0.0)
            print(f'{node.name} converted from Bipolar to Binary\n{node}')

MultiThreshold_24 converted from Bipolar to Binary
input: "Add_4_out0"
input: "MultiThreshold_24_param0"
output: "MultiThreshold_24_out0"
name: "MultiThreshold_24"
op_type: "MultiThreshold"
attribute {
  name: "out_dtype"
  s: "BINARY"
  type: STRING
}
attribute {
  name: "out_scale"
  f: 1.0
  type: FLOAT
}
attribute {
  name: "out_bias"
  f: 0.0
  type: FLOAT
}
domain: "qonnx.custom_op.general"



In [75]:
# global_out_name = model.graph.output[0].name
# global_out_name

In [76]:
# model.set_tensor_datatype(global_out_name, DataType["BINARY"])

In [77]:
# finn_bipolar_to_binary = models_folder + '/05_finn_bipolar_to_binary.onnx'

In [78]:
# model.save(finn_bipolar_to_binary)

In [79]:
# showInNetron(finn_bipolar_to_binary)

### Standlone Thresholds

In [80]:
# model = ModelWrapper(finn_bipolar_to_binary)

# model = model.transform(to_hw.InferThresholdingLayer())

In [81]:
# finn_std_alone_thres = models_folder + '/06_finn_std_alone_thres.onnx'

In [82]:
# model.save(finn_std_alone_thres)

In [83]:
# showInNetron(finn_std_alone_thres)

### Rest of the Streamline Process

In [84]:
# model = ModelWrapper(finn_streamline)

### Multithresholds to INT32

In [150]:
Multithreshold_node = model.get_nodes_by_op_type("MultiThreshold")    

for node in Multithreshold_node:
    if model.get_tensor_datatype(node.input[1]) == "FLOAT32":
        print(f'{node.name}: node with Float32 annotation')
        model.set_tensor_datatype(node.input[1], DataType["INT32"])
        print(f'{node.name}: changed to datatype {model.get_tensor_datatype(node.input[1])}')

MultiThreshold_1: node with Float32 annotation
MultiThreshold_1: changed to datatype INT32
MultiThreshold_6: node with Float32 annotation
MultiThreshold_6: changed to datatype INT32
MultiThreshold_14: node with Float32 annotation
MultiThreshold_14: changed to datatype INT32
MultiThreshold_21: node with Float32 annotation
MultiThreshold_21: changed to datatype INT32
MultiThreshold_24: node with Float32 annotation
MultiThreshold_24: changed to datatype INT32


In [151]:
def step_resnet50_convert_to_hw(model: ModelWrapper):
    # model.set_tensor_datatype(model.graph.input[0].name, DataType["UINT8"])
    model = model.transform(InferDataLayouts())
    model = model.transform(DoubleToSingleFloat())
    model = model.transform(InferDataTypes())
    model = model.transform(SortGraph())

    to_hw_transformations = [
        to_hw.InferAddStreamsLayer,
        LowerConvsToMatMul,
        to_hw.InferChannelwiseLinearLayer,
        # to_hw.InferPool,
        AbsorbTransposeIntoMultiThreshold,
        RoundAndClipThresholds,
        to_hw.InferQuantizedMatrixVectorActivation,
        to_hw.InferBinaryMatrixVectorActivation,
        to_hw.InferThresholdingLayer,
        AbsorbConsecutiveTransposes,
        to_hw.InferConvInpGen,
        to_hw.InferDuplicateStreamsLayer,
        to_hw.InferLabelSelectLayer,
    ]
    for trn in to_hw_transformations:
        model = model.transform(trn())
        model = model.transform(InferDataLayouts())
        model = model.transform(GiveUniqueNodeNames())
        model = model.transform(InferDataTypes())

    model = model.transform(RemoveCNVtoFCFlatten())
    model = model.transform(GiveReadableTensorNames())
    model = model.transform(RemoveUnusedTensors())
    model = model.transform(SortGraph())

    return model

In [152]:
model = step_resnet50_convert_to_hw(model)

AssertionError: MultiThreshold_0: Signed output requires actval < 0

In [86]:
# model = model.transform(to_hw.InferAddStreamsLayer())

# model = model.transform(to_hw.InferBinaryMatrixVectorActivation())
# model = model.transform(to_hw.InferQuantizedMatrixVectorActivation())
# #model = model.transform(to_hw.InferVectorVectorActivation()) # Only if DW convs

# # input quantization (if any) to standalone thresholding
# model = model.transform(to_hw.InferThresholdingLayer())

# model = model.transform(to_hw.InferConvInpGen())

InferenceError: [ShapeInferenceError] (op_type:RandomNormal, node name: Thresholding_MultiThreshold_6): [ShapeInferenceError] Inferred shape and existing shape differ in dimension 1: (32) vs (56)

In [70]:
finn_hw_layers = models_folder + '/07_fin_hw_layers.onnx'
model.save(finn_hw_layers)

In [71]:
showInNetron(finn_hw_layers)

Stopping http://0.0.0.0:8083
Serving './step_by_step_models/07_fin_hw_layers.onnx' at http://0.0.0.0:8083


In [ ]:
model = model.transform(to_hw.InferPool())
model = model.transform(to_hw.InferStreamingMaxPool())
model = model.transform(to_hw.InferConvInpGen())

# get rid of Reshape(-1, 1) operation between hw nodes 
model = model.transform(RemoveCNVtoFCFlatten())

# get rid of Tranpose -> Tranpose identity seq
model = model.transform(absorb.AbsorbConsecutiveTransposes())

# infer tensor data layouts
model = model.transform(InferDataLayouts())

model = model.transform(Streamline())

In [40]:
finn_hw_layers = models_folder + '/07_fin_hw_layers.onnx'
model.save(finn_hw_layers)

In [41]:
showInNetron(finn_hw_layers)

Stopping http://0.0.0.0:8083
Serving './step_by_step_models/07_fin_hw_layers.onnx' at http://0.0.0.0:8083


# Dataflow Partition

In [42]:
model = ModelWrapper(finn_hw_layers)
parent_model = model.transform(CreateDataflowPartition())

In [43]:
finn_parent_filename = models_folder + '/00_finn_dataflow_parent.onnx'
parent_model.save(finn_parent_filename)

In [44]:
showInNetron(finn_parent_filename)

Stopping http://0.0.0.0:8083
Serving './step_by_step_models/00_finn_dataflow_parent.onnx' at http://0.0.0.0:8083


In [45]:
sdp_node = parent_model.get_nodes_by_op_type("StreamingDataflowPartition")[0]
sdp_node = getCustomOp(sdp_node)
dataflow_filename = sdp_node.get_nodeattr("model")
dataflow_model = ModelWrapper(dataflow_filename)

# Specialize Layers

In [46]:
from finn.transformation.fpgadataflow.specialize_layers import SpecializeLayers

### Change Padding Nodes to HLS, so Auto Folding can be applied

In [47]:
FMPadding_node = dataflow_model.get_nodes_by_op_type("FMPadding")

for node in FMPadding_node:
    node_inst = getCustomOp(node)
    node_inst.set_nodeattr("preferred_impl_style", "hls")
    print(f'Node {node.name} forced to HLS')

Node FMPadding_0 forced to HLS
Node FMPadding_1 forced to HLS
Node FMPadding_2 forced to HLS
Node FMPadding_3 forced to HLS
Node FMPadding_4 forced to HLS
Node FMPadding_5 forced to HLS
Node FMPadding_6 forced to HLS
Node FMPadding_7 forced to HLS
Node FMPadding_8 forced to HLS
Node FMPadding_9 forced to HLS
Node FMPadding_10 forced to HLS


In [48]:
# save the dataflow partition with a different name for easier access
# and specialize the layers to HLS variants
dataflow_model = dataflow_model.transform(SpecializeLayers(fpga_part))

dataflow_model = dataflow_model.transform(GiveUniqueNodeNames())
dataflow_model = dataflow_model.transform(GiveReadableTensorNames())

finn_dataflow_filename = models_folder + '/20_finn_dataflow_model.onnx'
dataflow_model.save(finn_dataflow_filename)

In [49]:
showInNetron(finn_dataflow_filename)

Stopping http://0.0.0.0:8083
Serving './step_by_step_models/20_finn_dataflow_model.onnx' at http://0.0.0.0:8083


### Check execution???

In [50]:
# parent_dataflow_model = ModelWrapper(finn_parent_filename)

In [51]:
# input_dict = {"global_in": test_ip*255}
# output_dict = oxe.execute_onnx(parent_dataflow_model, input_dict)
# produced_clean_qonnx = output_dict[list(output_dict.keys())[0]]
# produced_clean_qonnx

# Folding Factors

In [52]:
from finn.transformation.fpgadataflow.set_folding import SetFolding

**Taregt Cycles Per Frame**

If target is 25 FPS, inference time is $\frac{1}{25}=40ms$

If $clk = 10 ns$:
$$
Target~Cycles~Per~Frame = \frac{40\times 10^{-3}}{10\times 10^{-9}}= 4\times 10^{6}
$$

No se tiene en cuenta el tiempo de preprocesado, que en realidad debería ser inexistente, ya que está embebido en el preprocess del modelo. 

In [53]:
model = ModelWrapper(finn_dataflow_filename)

apply method of SetFolding returns (model, False), so model is [0]

maybe it is easier to do: model, _ = folder.apply(...)

In [54]:
model = model.transform(SetFolding(
    target_cycles_per_frame=4000000,
    mvau_wwidth_max=36,
    two_pass_relaxation=False)
)

In [55]:
folding_filename = models_folder + '/30_finn_folding.onnx'
#model[0].save(folding_filename)
model.save(folding_filename)

In [56]:
showInNetron(folding_filename)

Stopping http://0.0.0.0:8083
Serving './step_by_step_models/30_finn_folding.onnx' at http://0.0.0.0:8083


### Check Total Estimated Cycles, looping over each node attribute

In [57]:
all_nodes = model.get_finn_nodes()

In [58]:
i = 0
total_cycles = []
for node in all_nodes:
    my_node = getCustomOp(node)
    node_cycles = my_node.get_nodeattr("cycles_estimate")
    total_cycles.append(node_cycles)
    print(f'Node {i} estimated cycles: {node_cycles}')
    i += 1
print(f'\nTotal estimated cycles: {np.array(total_cycles).sum()}')

Node 0 estimated cycles: 150528
Node 1 estimated cycles: 153228
Node 2 estimated cycles: 340053
Node 3 estimated cycles: 3612672
Node 4 estimated cycles: 401408
Node 5 estimated cycles: 415872
Node 6 estimated cycles: 3620064
Node 7 estimated cycles: 3612672
Node 8 estimated cycles: 401408
Node 9 estimated cycles: 3211264
Node 10 estimated cycles: 100352
Node 11 estimated cycles: 1605632
Node 12 estimated cycles: 200704
Node 13 estimated cycles: 207936
Node 14 estimated cycles: 504144
Node 15 estimated cycles: 451584
Node 16 estimated cycles: 50176
Node 17 estimated cycles: 802816
Node 18 estimated cycles: 50176
Node 19 estimated cycles: 1605632
Node 20 estimated cycles: 100352
Node 21 estimated cycles: 107648
Node 22 estimated cycles: 906976
Node 23 estimated cycles: 903168
Node 24 estimated cycles: 100352
Node 25 estimated cycles: 1605632
Node 26 estimated cycles: 50176
Node 27 estimated cycles: 1605632
Node 28 estimated cycles: 100352
Node 29 estimated cycles: 107648
Node 30 estimat

# Minimize 

In [59]:
from finn.transformation.fpgadataflow.minimize_accumulator_width import (
    MinimizeAccumulatorWidth,
)
from finn.transformation.fpgadataflow.minimize_weight_bit_width import (
    MinimizeWeightBitWidth,
)

In [60]:
model = ModelWrapper(folding_filename)

In [61]:
model = model.transform(MinimizeAccumulatorWidth())
model = model.transform(MinimizeWeightBitWidth())

/home/gmoreno/uav/finn/src/finn/custom_op/fpgadataflow/thresholding.py:84: UserWarning: inputDataType changing for Thresholding_rtl_1: INT32 -> INT15 
  warnings.warn(warn_str)
/home/gmoreno/uav/finn/src/finn/custom_op/fpgadataflow/thresholding.py:84: UserWarning: inputDataType changing for Thresholding_rtl_2: INT32 -> INT10 
  warnings.warn(warn_str)
/home/gmoreno/uav/finn/src/finn/custom_op/fpgadataflow/thresholding.py:84: UserWarning: inputDataType changing for Thresholding_rtl_3: INT32 -> INT12 
  warnings.warn(warn_str)
/home/gmoreno/uav/finn/src/finn/custom_op/fpgadataflow/thresholding.py:84: UserWarning: inputDataType changing for Thresholding_rtl_4: INT32 -> INT10 
  warnings.warn(warn_str)
/home/gmoreno/uav/finn/src/finn/custom_op/fpgadataflow/thresholding.py:84: UserWarning: inputDataType changing for Thresholding_rtl_5: INT32 -> INT11 
  warnings.warn(warn_str)
/home/gmoreno/uav/finn/src/finn/custom_op/fpgadataflow/thresholding.py:84: UserWarning: inputDataType changing for 

In [62]:
minimize_filename = models_folder + '/31_finn_minimize.onnx'
model.save(minimize_filename)

In [63]:
showInNetron(minimize_filename)

Stopping http://0.0.0.0:8083
Serving './step_by_step_models/31_finn_minimize.onnx' at http://0.0.0.0:8083


# HW IP Generation: PrepareIP and HLSSynthIP 

In [64]:
# from finn.transformation.fpgadataflow.prepare_ip import PrepareIP
# from finn.transformation.fpgadataflow.hlssynth_ip import HLSSynthIP

In [65]:
# model = ModelWrapper(minimize_filename)

In [66]:
# model = model.transform(PrepareIP(fpga_part, target_clk_ns))
# model = model.transform(HLSSynthIP())

In [67]:
# hw_filename = models_folder + '32_finn_hw_ipgen.onnx'
# model.save(hw_filename)

In [68]:
# showInNetron(hw_filename)

# FIFO depths

In [69]:
from finn.transformation.fpgadataflow.set_fifo_depths import InsertAndSetFIFODepths

In [70]:
#model = ModelWrapper(hw_filename)

# model = ModelWrapper(minimize_filename)

In [71]:
# model = model.transform(InsertAndSetFIFODepths(
#     fpgapart=fpga_part,
#     clk_ns=10.0,
#     max_qsrl_depth=256,
#     max_depth=None,
#     swg_exception=False,#True, # Used to optimize convolution FIFOs, splitting in several with Power of Two
#     vivado_ram_style="auto",
#     force_python_sim=False)
# )

In [72]:
# fifo_filename = models_folder + '31_finn_fifo.onnx'
# #model[0].save(fifo_filename)
# model.save(fifo_filename)

In [73]:
# showInNetron(fifo_filename)

### Streamline FIFOs

In [74]:
from finn.transformation.fpgadataflow.set_fifo_depths import SplitLargeFIFOs
from finn.transformation.fpgadataflow.set_fifo_depths import RemoveShallowFIFOs

In [75]:
#model = model[0].transform(SplitLargeFIFOs())

# model = model.transform(SplitLargeFIFOs())
# model = model.transform(RemoveShallowFIFOs())

In [76]:
# after FIFOs are ready to go, call PrepareIP and HLSSynthIP again
# this will only run for the new nodes (e.g. FIFOs and DWCs) -> DWCs for Mobilenet
# model = model.transform(PrepareIP(fpga_part, target_clk_ns))
# model = model.transform(HLSSynthIP())

In [77]:
# fifo_streamline_filename = models_folder + '33_finn_fifo_streamline.onnx'
# model.save(fifo_streamline_filename)

In [78]:
# showInNetron(fifo_streamline_filename)

# PYNQ Driver

In [79]:
# from finn.transformation.fpgadataflow.make_zynq_proj import ZynqBuild

In [80]:
# model = ModelWrapper(fifo_streamline_filename)
# model = model.transform(ZynqBuild(platform = pynq_board, period_ns = target_clk_ns))

In [81]:
# from finn.transformation.fpgadataflow.make_pynq_driver import MakePYNQDriver

In [82]:
# model = model.transform(MakePYNQDriver("zynq-iodma"))

In [83]:
# pynq_driver_filename = '/40_pynq_driver.onnx'
# model.save(pynq_driver_filename)